### Importing the dataset

### Data cleaning

### Encoding Categorical Data

In [ ]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building the ANN model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 54, kernel_initializer = 'uniform', activation = 'relu', input_dim = 107))

# Adding the next five hidden layers
for i in range(3):
    classifier.add(Dense(units = 54, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p=0.1))
    
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fitting the ANN normally

In [ ]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

### Fitting the ANN with GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'batch size' : [10,20,30]
              'epochs' : [100,200,300]
              'optimizer' : ['adam','adam','adam']}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', CV = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

### Predicting new observation

In [ ]:
new_data = []
new_pred = classifier.predict(sc.transform([[new_data]]))